In [1]:
import openai

In [2]:
client = openai.Client()

In [3]:
assistant = client.beta.assistants.create(
    name="Tutor de Tecnologia",
    instructions="Você é um tutor sobre assuntos relacionados a tecnologia",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-0125",
)

In [5]:
pergunta = """
    Em um sistema de cache com mapeamento direto, dado uma memória principal de uma cache de 256 KB e blocos de 64 bytes:
        - Quantas linhas existem na cache?
        - Onde o bloco de endereço 0x1A23C será mapeado na cache?
"""

In [6]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

C:\Users\kaues\AppData\Local\Temp\ipykernel_15672\832815408.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\kaues\AppData\Local\Temp\ipykernel_15672\832815408.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [7]:
# Executa a thread 
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

C:\Users\kaues\AppData\Local\Temp\ipykernel_15672\209825715.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [8]:
run.status

'queued'

In [9]:
# Aguarda a thread rodar
import time

while run.status in ["queued", "in_progress", "cancellig"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

C:\Users\kaues\AppData\Local\Temp\ipykernel_15672\435338251.py:6: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [10]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

C:\Users\kaues\AppData\Local\Temp\ipykernel_15672\184256488.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_N8Tz41pzgJQCR2Uyk5YQjq9l', assistant_id='asst_Cswo3wIti8u3BWoXb0NrTPBo', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O bloco de endereço 0x1A23C será mapeado na linha (índice) 1672 da cache, com um deslocamento (offset) de 60 bytes. A tag para este bloco será 0.\n\nSe precisar de mais alguma coisa, não hesite em me chamar.'), type='text')], created_at=1749764284, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_xcXlNEHWwQ5Fot04KD5iMWj8', status=None, thread_id='thread_57XxpTgalmcIOAJwYoxdRekX'), Message(id='msg_iy0iIUlmLioWEf7EcX45O85G', assistant_id='asst_Cswo3wIti8u3BWoXb0NrTPBo', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Parece que houve um erro de tipo na operação bitwise "&". Vamos corrigir isso convertendo o número de linhas para um inteiro antes de realizar a operação. V

In [11]:
print(mensagens.data[0].content[0].text.value)

O bloco de endereço 0x1A23C será mapeado na linha (índice) 1672 da cache, com um deslocamento (offset) de 60 bytes. A tag para este bloco será 0.

Se precisar de mais alguma coisa, não hesite em me chamar.


In [12]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

C:\Users\kaues\AppData\Local\Temp\ipykernel_15672\1597243276.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [13]:
for step in run_steps.data[::-1]:
    print(f"\n=====Step {step.step_details.type}")

    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" * 10)
            print(tool_call.code_interpreter.input)
            print("=" * 10)

    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )

        print(message.content[0].text.value)


=====Step message_creation


C:\Users\kaues\AppData\Local\Temp\ipykernel_15672\2963295479.py:11: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Para determinar o número de linhas na cache e onde o bloco de endereço 0x1A23C será mapeado na cache, precisamos fazer alguns cálculos com base nas informações fornecidas.

Dado que temos uma cache com mapeamento direto, o endereço da memória principal é dividido em três partes: tag, conjunto e offset. O offset é determinado pelo tamanho do bloco (64 bytes), o conjunto é determinado pela quantidade de linhas na cache e a tag é o restante.

Vamos começar calculando o número de linhas na cache.

=====Step tool_calls
# Tamanho da cache em bytes
cache_size = 256 * 1024  # 256 KB em bytes

# Tamanho do bloco em bytes
block_size = 64

# Calculando o número de linhas na cache (conjuntos)
num_lines = cache_size / block_size

num_lines

=====Step message_creation
A cache possui 4096 linhas.

Agora, vamos determinar onde o bloco de endereço 0x1A23C será mapeado na cache. Para isso, precisamos analisar a estrutura do endereço de memória:
- O offset é determinado pelos bits menos significativos, d